<!--
 * @file        3_gnss.ipynb
 * @brief       This notebook provides GNSS data processing and analysis.
 *
 * @authors     Jaehwan Lee (idljh5529@gmail.com)
 *
 * @date        2025-08-11 Released by AI Lab, Hansung University
 *
-->

# 3. GNSS (Global Navigation Satellite System)

이번 실습에서는 자율주행 차량에서 핵심적인 역할을 하는 GNSS 시스템에 대해 학습합니다.

## 실습 목표
1. **NMEA 데이터 분석**: NMEA 로그 파일에서 GGA와 RMC 메시지를 파싱하고 분석합니다.
2. **위성 지도 시각화**: GPS 궤적을 실제 위성 지도 위에 시각화합니다.
3. **ROS bag GNSS 데이터 분석**: 직접 취득한 GNSS 데이터를 분석합니다.
4. **좌표 변환**: WGS84와 ENU 좌표계 간 변환을 구현하고 검증합니다.

## 다루는 GNSS 데이터
- **NMEA GGA**: 위치, 고도, HDOP, 위성 수 등 기본 위치 정보
- **NMEA RMC**: 위치, 속도, 방향, 상태 등 항법 정보
- **직접 취득한 GPS 데이터**: 위치, 속도 정보 및 정확도 지표
  - 위치 정보: latitude, longitude, altitude
  - 정확도 지표: DOP 값들 (GDOP, PDOP, HDOP, VDOP, TDOP)
  - 오차 정보: err_horz, err_vert, position_covariance
  - 운동 정보: speed, track (heading)

## GNSS 실습 필요성

### 1. 자율주행에서 GNSS의 역할
- **절대 위치 추정**: 차량의 정확한 지구상 위치 파악
- **경로 계획**: 목적지까지의 최적 경로 생성
- **맵 매칭**: 디지털 지도와 실제 위치 연계
- **센서 퓨전**: IMU와 같은 다른 센서들과 결합하여 정확도 향상

### 2. GNSS 데이터 품질 평가
- **DOP 분석**: 위성 배치에 따른 정확도 평가
- **공분산 행렬**: 위치 불확실성 정보 평가
- **위성 상태 모니터링**: 가용 위성 수 분석
- **실시간 품질 모니터링**: 연속적인 정확도 평가

### 3. 좌표계 변환의 중요성
- **지역 좌표계 활용**: 차량 주변 환경을 직관적으로 표현
- **센서 통합**: 다양한 센서 데이터를 일관된 좌표계에서 처리
- **정밀 제어**: 상대적 위치 기반 정밀 차량 제어

## 실습 데이터
1. **NMEA 로그 파일**: 표준 GPS 수신기 출력 데이터
2. **ROS bag 파일**: 직접 취득한 `/gps/gps` 토픽의 GPS Fix 데이터

## GPS Fix 메시지 주요 필드 설명

### 위치 정보
- `latitude`, `longitude`: WGS84 좌표계 기준 위치 (도 단위)
- `altitude`: 해수면 기준 고도 (미터)
- `track`: 지북 기준 진행 방향 (도, 0~360°)
- `speed`: 지상 속도 (m/s)

### 정확도 지표 (DOP: Dilution of Precision)
- `GDOP`: 기하학적 정밀도 저하 (전체적 정확도)
- `PDOP`: 위치 정밀도 저하 (3차원 위치)
- `HDOP`: 수평 정밀도 저하 (2차원 위치)
- `VDOP`: 수직 정밀도 저하 (고도)
- `TDOP`: 시간 정밀도 저하

### 오차 정보
- `err_horz`: 수평 위치 오차 (m)
- `err_vert`: 수직 위치 오차 (m)
- `position_covariance`: 3×3 위치 공분산 행렬

### 위성 정보
- `satellites_used`: 사용 중인 위성 수
- `satellites_visible`: 사용 가능한 위성 수

In [ ]:
# 외부에 정의된 파이썬 모듈(.py 파일)을 수정할 때마다 매번 커널을 재시작하지 않아도 변경 사항이 자동으로 반영되도록 설정
%load_ext autoreload
%autoreload 2

In [ ]:
# 필요한 라이브러리 임포트
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 사용자 정의 모듈 임포트
sys.path.append('./tutlibs/gnss')
from tutlibs.gnss.custom_gnss import GNSSProcessor

print("라이브러리 임포트 완료!")
print("Python 버전:", sys.version)
print("NumPy 버전:", np.__version__)
print("Matplotlib 버전:", plt.matplotlib.__version__)
print("Pandas 버전:", pd.__version__)


## 1. NMEA 데이터 분석

NMEA (National Marine Electronics Association) 0183은 GPS 수신기에서 가장 널리 사용되는 표준 통신 프로토콜입니다. 
이번 실습에서는 GGA와 RMC 메시지를 중심으로 NMEA 데이터를 분석합니다.

### NMEA 메시지 종류
- **GGA (Global Positioning System Fix Data)**: 위치, 고도, HDOP, 위성 수 등
- **RMC (Recommended Minimum)**: 위치, 속도, 방향, 날짜, 상태 등

### 분석 항목
- 궤적 시각화
- HDOP (Horizontal Dilution of Precision) 변화
- 위성 수 변화  
- 속도 프로파일
- 데이터 품질 통계


In [ ]:
# GNSS 처리 객체 생성
gnss_processor = GNSSProcessor()

# NMEA 파일 경로 설정 (실제 파일이 없으면 시뮬레이션 데이터 사용)
nmea_file_path = "./../data/gnss/gps_nmea_AMOD_AGL3080_20121104_134730.txt"

print("=== NMEA 데이터 로딩 ===")
print(f"파일 경로: {nmea_file_path}")

# NMEA 데이터 파싱
nmea_data = gnss_processor.parse_nmea_file(nmea_file_path) # TODO: parse_nmea_file 함수 완성

print(f"✓ NMEA 데이터 로딩 완료")
print(f"GGA 메시지 수: {len(nmea_data['gga'])}")
print(f"RMC 메시지 수: {len(nmea_data['rmc'])}")

# 첫 번째 GGA와 RMC 메시지 샘플 출력
if nmea_data['gga']:
    print(f"\n=== GGA 메시지 샘플 ===")
    sample_gga = nmea_data['gga'][0]
    for key, value in sample_gga.items():
        print(f"{key}: {value}")

if nmea_data['rmc']:
    print(f"\n=== RMC 메시지 샘플 ===")
    sample_rmc = nmea_data['rmc'][0]
    for key, value in sample_rmc.items():
        print(f"{key}: {value}")


In [ ]:
# NMEA 데이터 종합 분석 및 시각화
print("=== NMEA 데이터 분석 시각화 ===")

gnss_processor.plot_nmea_analysis(nmea_data)

# 개별 데이터 품질 분석
gga_data = nmea_data['gga']
rmc_data = nmea_data['rmc']

if gga_data:
    print("\n=== GGA 데이터 품질 분석 ===")
    
    # Fix Quality 분석
    fix_qualities = [d['fix_quality'] for d in gga_data]
    fix_quality_counts = {}
    for fq in fix_qualities:
        fix_quality_counts[fq] = fix_quality_counts.get(fq, 0) + 1
    
    print("Fix Quality 분포:")
    for quality, count in fix_quality_counts.items():
        quality_name = {0: "Invalid", 1: "GPS", 2: "DGPS", 3: "PPS", 
                       4: "Real Time Kinematic", 5: "Float RTK", 6: "Estimated"}.get(quality, "Unknown")
        print(f"  {quality} ({quality_name}): {count}회 ({count/len(gga_data)*100:.1f}%)")
    
    # HDOP 통계
    hdops = [d['hdop'] for d in gga_data if d['hdop'] > 0]
    if hdops:
        print(f"\nHDOP 통계:")
        print(f"  평균: {np.mean(hdops):.2f}")
        print(f"  최소: {min(hdops):.2f}")
        print(f"  최대: {max(hdops):.2f}")
        print(f"  표준편차: {np.std(hdops):.2f}")

if rmc_data:
    print("\n=== RMC 데이터 품질 분석 ===")
    
    # Status 분석
    statuses = [d['status'] for d in rmc_data]
    status_counts = {}
    for status in statuses:
        status_counts[status] = status_counts.get(status, 0) + 1
    
    print("Navigation Status 분포:")
    for status, count in status_counts.items():
        status_name = {"A": "Active (Valid)", "V": "Void (Invalid)"}.get(status, "Unknown")
        print(f"  {status} ({status_name}): {count}회 ({count/len(rmc_data)*100:.1f}%)")
    
    # 속도 통계
    speeds = [d['speed_knots'] for d in rmc_data if d['speed_knots'] is not None]
    if speeds:
        print(f"\n속도 통계:")
        print(f"  평균: {np.mean(speeds):.2f} knots ({np.mean(speeds)*1.852:.2f} km/h)")
        print(f"  최대: {max(speeds):.2f} knots ({max(speeds)*1.852:.2f} km/h)")
        print(f"  최소: {min(speeds):.2f} knots ({min(speeds)*1.852:.2f} km/h)")
        print(f"  표준편차: {np.std(speeds):.2f} knots ({np.std(speeds)*1.852:.2f} km/h)")


## 2. 위성 지도 시각화

NMEA 데이터로부터 추출한 GPS 궤적을 실제 위성 지도 위에 시각화합니다.
이를 통해 차량의 실제 주행 경로를 직관적으로 확인할 수 있습니다.

### 시각화하는 정보들
- 실제 위성 지도 배경
- 시간에 따른 색상 변화 (jet colormap)
- 시작점과 끝점 마커
- 헤딩 방향 화살표 (가능한 경우)

**Note**: 지도 시각화가 표시되지 않을 경우, 인터넷 연결을 확인하세요.

In [ ]:
# 위성 지도 시각화
print("=== 위성 지도 시각화 ===")

# GGA 데이터에서 유효한 좌표 추출
valid_gga = [d for d in nmea_data['gga'] if d['latitude'] is not None and d['longitude'] is not None]

if valid_gga:
    latitudes = [xxxxxx for d in valid_gga] # TODO: 위도 추출
    longitudes = [xxxxxx for d in valid_gga] # TODO: 경도 추출
    
    # 시간 정보 생성 (실제 시간이 없으므로 인덱스 기반)
    times = list(range(len(latitudes)))
    
    # RMC에서 헤딩 정보 추출 (가능한 경우)
    headings = None
    valid_rmc = [d for d in nmea_data['rmc'] if d['course'] is not None and d['course'] > 0]
    if valid_rmc and len(valid_rmc) >= len(valid_gga) * 0.8:  # 80% 이상 유효한 경우만
        headings = [d['course'] for d in valid_rmc[:len(valid_gga)]]
    
    print(f"유효한 좌표 포인트: {len(latitudes)}개")
    print(f"위도 범위: {min(latitudes):.6f} ~ {max(latitudes):.6f}")
    print(f"경도 범위: {min(longitudes):.6f} ~ {max(longitudes):.6f}")
    
    # 위성 지도 생성
    try:
        satellite_map = gnss_processor.create_satellite_map(
            xxxxxx,
            xxxxxx,
            xxxxxx,
            xxxxxx,
            title="NMEA GPS Trajectory on Satellite Map"
        ) # TODO: create_satellite_map 함수 완성
        
        if satellite_map:
            print("✓ 위성 지도 생성 완료")
            print("지도를 보려면 다음 셀을 실행하세요:")
            print("satellite_map")
        else:
            print("⚠ 위성 지도 생성 실패")
            
    except Exception as e:
        print(f"⚠ 위성 지도 생성 중 오류 발생: {e}")
        print("대신 기본 matplotlib 플롯을 표시합니다.")
        
        # 기본 플롯으로 대체
        plt.figure(figsize=(12, 8))
        plt.plot(longitudes, latitudes, 'b-', linewidth=2)
        plt.plot(longitudes[0], latitudes[0], 'go', markersize=10, label='Start')
        plt.plot(longitudes[-1], latitudes[-1], 'ro', markersize=10, label='End')
        
        # 헤딩 화살표 추가 (있는 경우)
        if headings:
            step = max(1, len(latitudes) // 20)
            for i in range(0, len(latitudes), step):
                if i < len(headings):
                    dx = 0.00001 * np.sin(np.deg2rad(headings[i]))
                    dy = 0.00001 * np.cos(np.deg2rad(headings[i]))
                    plt.arrow(longitudes[i], latitudes[i], dx, dy, 
                             head_width=0.000005, head_length=0.000005, 
                             fc='black', ec='black')
        
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title('GPS Trajectory (NMEA Data)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.axis('equal')
        plt.tight_layout()
        plt.show()
        
else:
    print("⚠ 유효한 GPS 좌표가 없습니다.")


In [ ]:
# GNSS 데이터와 위성 사진 시각화
satellite_map

## 3. 실제 취득한 GPS Fix 데이터 분석 (`/gps/gps` 토픽)

이번에는 ROS bag 파일에서 `/gps/gps` 토픽의 `gps_common/GPSFix` 메시지를 분석합니다.
이 메시지는 NMEA보다 더 상세한 정보를 제공하며, 자율주행에 필요한 많은 정보가 통합되어 있습니다.

### GPS Fix 메시지의 특징
- **통합 데이터**: 위치, 속도, 정확도, 상태 정보가 하나의 메시지에 포함
- **DOP 값들**: 5가지 DOP (GDOP, PDOP, HDOP, VDOP, TDOP) 제공
- **공분산 행렬**: 위치 불확실성의 전체 정보 (9개 원소)
- **실시간 상태**: 위성 수, 수신기 상태 등

### 분석할 주요 필드
- `latitude`, `longitude`, `altitude`: 기본 위치 정보
- `speed`, `track`: 속도 및 진행 방향
- `gdop`, `pdop`, `hdop`, `vdop`, `tdop`: 정밀도 저하 지표
- `err_horz`, `err_vert`: 예상 위치 오차
- `position_covariance`: 위치 공분산 행렬
- `satellites_used`, `satellites_visible`: 위성 정보

In [ ]:
# ROS bag GPS Fix 데이터 로딩
print("=== ROS bag GPS Fix 데이터 로딩 ===")

# ROS bag 파일 경로 설정
bag_file_path = "./../data/imu/3_overtaking.bag"

# 로드할 토픽 정의
gps_topics = ['/gps/gps']

print(f"파일 경로: {bag_file_path}")
print(f"분석할 토픽: {gps_topics}")

# GPS Fix 데이터 로딩
rosbag_gps_data = gnss_processor.load_rosbag_gnss_data(xxxxxx, xxxxxx) # TODO: load_rosbag_gnss_data 함수 인자 입력

print(f"\n✓ ROS bag GPS Fix 데이터 로딩 완료")
for topic in gps_topics:
    if topic in rosbag_gps_data:
        data_count = len(rosbag_gps_data[topic]['data'])
        time_span = 0
        if data_count > 1:
            times = rosbag_gps_data[topic]['time']
            time_span = times[-1] - times[0]
        print(f"{topic}: {data_count}개 메시지, {time_span:.1f}초")

# 첫 번째 GPS Fix 메시지 샘플 출력
print(f"\n=== GPS Fix 메시지 샘플 ===")
if '/gps/gps' in rosbag_gps_data and rosbag_gps_data['/gps/gps']['data']:
    sample_gps = rosbag_gps_data['/gps/gps']['data'][0]
    print(f"위치 정보:")
    print(f"  Latitude: {sample_gps.latitude:.8f}°")
    print(f"  Longitude: {sample_gps.longitude:.8f}°") 
    print(f"  Altitude: {sample_gps.altitude:.3f} m")
    print(f"  Speed: {sample_gps.speed:.3f} m/s")
    print(f"  Track: {sample_gps.track:.1f}°")
    
    print(f"\nDOP 값들:")
    print(f"  GDOP: {sample_gps.gdop:.2f}")
    print(f"  PDOP: {sample_gps.pdop:.2f}")
    print(f"  HDOP: {sample_gps.hdop:.2f}")
    print(f"  VDOP: {sample_gps.vdop:.2f}")
    print(f"  TDOP: {sample_gps.tdop:.2f}")
    
    print(f"\n오차 정보:")
    print(f"  Horizontal Error: {sample_gps.err_horz:.3f} m")
    print(f"  Vertical Error: {sample_gps.err_vert:.3f} m")
    
    print(f"\n위성 정보:")
    print(f"  Satellites Used: {sample_gps.status.satellites_used}")
    print(f"  Satellites Visible: {sample_gps.status.satellites_visible}")
    
    print(f"\n공분산 정보:")
    print(f"  Position Covariance Type: {sample_gps.position_covariance_type}")
    print(f"  Covariance Matrix (대각 성분):")
    print(f"    σ²_xx (Lon): {sample_gps.position_covariance[0]:.6f}")
    print(f"    σ²_yy (Lat): {sample_gps.position_covariance[4]:.6f}")
    print(f"    σ²_zz (Alt): {sample_gps.position_covariance[8]:.6f}")

In [ ]:
# GPS Fix 데이터 종합 분석 및 시각화
print("=== GPS Fix 데이터 종합 분석 시각화 ===")

gnss_processor.plot_gps_fix_analysis(rosbag_gps_data)


## 4. 위성 지도 시각화

GPS Fix 데이터로부터 추출한 궤적을 실제 위성 지도 위에 시각화합니다.
헤딩 정보와 시간에 따른 색상 변화를 포함하여 차량의 실제 주행 패턴을 직관적으로 확인할 수 있습니다.

### 시각화 특징
- 실제 위성 지도 배경 (ArcGIS World Imagery)
- 시간에 따른 색상 변화 (jet colormap)
- 차량 헤딩 방향 화살표
- 시작점과 끝점 마커

**Note**: 지도 시각화가 표시되지 않을 경우, 인터넷 연결을 확인하세요.

In [ ]:
# GPS Fix 데이터로 위성 지도 시각화
print("=== GPS Fix 궤적 위성 지도 시각화 ===")

if '/gps/gps' in rosbag_gps_data:
    gps_data = rosbag_gps_data['/gps/gps']
    
    if gps_data['data']:
        # GPS Fix 데이터에서 궤적 및 헤딩 추출
        latitudes = [msg.latitude for msg in gps_data['data']]
        longitudes = [msg.longitude for msg in gps_data['data']]
        times = np.array(gps_data['time'])
        relative_times = times - times[0]
        headings = [msg.track for msg in gps_data['data']]
        
        print(f"GPS Fix 좌표 포인트: {len(latitudes)}개")
        print(f"위도 범위: {min(latitudes):.8f}° ~ {max(latitudes):.8f}°")
        print(f"경도 범위: {min(longitudes):.8f}° ~ {max(longitudes):.8f}°")
        print(f"헤딩 범위: {min(headings):.1f}° ~ {max(headings):.1f}°")
        print(f"시간 범위: {relative_times[-1]:.1f}초")
        
        # 위성 지도 생성 (헤딩 포함)
        try:
            gps_satellite_map = gnss_processor.create_satellite_map(
                latitudes=latitudes,
                longitudes=longitudes,
                times=relative_times,
                headings=headings,
                title="GPS Fix Trajectory with Heading"
            ) # TODO: create_satellite_map 함수 완성
            
            if gps_satellite_map:
                print("✓ GPS Fix 위성 지도 생성 완료")
                print("지도를 보려면 다음 셀을 실행하세요:")
                print("gps_satellite_map")
            else:
                print("⚠ 위성 지도 생성 실패")
                
        except Exception as e:
            print(f"⚠ 위성 지도 생성 중 오류 발생: {e}")
            print("대신 기본 matplotlib 플롯을 표시합니다.")
            
            # 기본 플롯으로 대체 (헤딩 화살표 포함)
            plt.figure(figsize=(14, 10))
            
            # 시간에 따른 색상 변화
            colors = plt.cm.jet(relative_times / relative_times[-1])
            
            plt.subplot(2, 1, 1)
            for i in range(len(latitudes)-1):
                plt.plot([longitudes[i], longitudes[i+1]], [latitudes[i], latitudes[i+1]], 
                        color=colors[i], linewidth=2)
            
            plt.plot(longitudes[0], latitudes[0], 'go', markersize=12, label='Start')
            plt.plot(longitudes[-1], latitudes[-1], 'ro', markersize=12, label='End')
            
            # 헤딩 화살표 추가
            step = max(1, len(latitudes) // 20)
            for i in range(0, len(latitudes), step):
                dx = 0.00002 * np.sin(np.deg2rad(headings[i]))
                dy = 0.00002 * np.cos(np.deg2rad(headings[i]))
                plt.arrow(longitudes[i], latitudes[i], dx, dy, 
                         head_width=0.00001, head_length=0.00001, 
                         fc='black', ec='black', linewidth=1)
            
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.title('GPS Fix Trajectory with Heading Arrows')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.axis('equal')
            
            # 시간별 색상 바
            plt.subplot(2, 1, 2)
            scatter = plt.scatter(relative_times, [0]*len(relative_times), 
                                c=relative_times, cmap='jet', s=50)
            plt.colorbar(scatter, label='Time (seconds)')
            plt.xlabel('Time (seconds)')
            plt.title('Time Colormap Reference')
            plt.yticks([])
            
            plt.tight_layout()
            plt.show()
        
        # DOP와 오차의 상관관계 분석
        hdops = [msg.hdop for msg in gps_data['data']]
        err_horzs = [msg.err_horz for msg in gps_data['data']]
        speeds = [msg.speed for msg in gps_data['data']]
        
        print(f"\n=== GPS Fix 데이터 특성 분석 ===")
        print(f"평균 속도: {np.mean(speeds):.2f} m/s ({np.mean(speeds)*3.6:.1f} km/h)")
        print(f"최대 속도: {np.max(speeds):.2f} m/s ({np.max(speeds)*3.6:.1f} km/h)")
        print(f"평균 HDOP: {np.mean(hdops):.2f}")
        print(f"평균 수평 오차: {np.mean(err_horzs):.3f} m")
        print(f"HDOP와 수평 오차 상관계수: {np.corrcoef(hdops, err_horzs)[0,1]:.3f}")
        
else:
    print("⚠ GPS Fix 데이터를 찾을 수 없습니다.")

In [ ]:
# GNSS 데이터와 위성 사진 시각화
gps_satellite_map

## 4. 좌표 변환 실습 (WGS84 ↔ ENU)

자율주행에서는 전역 좌표계(WGS84)와 지역 좌표계(ENU) 간 변환이 필수적입니다.
이번 실습에서는 직접 구현한 좌표 변환 함수의 정확도를 검증합니다.

### 좌표계 설명
- **WGS84**: 세계 측지계, GPS가 사용하는 전역 좌표계 (위도, 경도, 고도)
- **ENU**: East-North-Up, 지역 직교 좌표계 (동쪽, 북쪽, 위쪽)

### 변환 과정
1. **WGS84 → ENU**: 글로벌 좌표를 로컬 좌표로 변환
2. **ENU → WGS84**: 로컬 좌표를 다시 글로벌 좌표로 역변환
3. **정확도 검증**: 원본과 재변환 결과 비교

### 변환 수식
**WGS84 → ENU:**
```
E = Δlon × (N + h) × cos(φ₀)
N = Δlat × (M + h)  
U = Δh
```

**ENU → WGS84:**
```
Δlat = N / (M + h)
Δlon = E / ((N + h) × cos(φ₀))
Δh = U
```

여기서 M은 자오선 곡률반지름, N은 수직 곡률반지름입니다.

### 4-1. WGS84 to ENU 좌표 변환

취득한 GPS 정보를 받아와 ENU 좌표계로 변환하는 실습을 진행합니다.

In [ ]:
### WGS84 데이터 로드 ###
# ROS bag 파일 경로 설정
bag_file_path = "./../data/gnss/hanyang_gps2.bag"

# 로드할 토픽 정의
gps_topics = ['/ublox_gps/fix']

print(f"파일 경로: {bag_file_path}")
print(f"분석할 토픽: {gps_topics}")

# GPS Fix 데이터 로딩
rosbag_gps_data = gnss_processor.load_rosbag_gnss_data(bag_file_path, gps_topics)

# GPS Fix 데이터에서 궤적 추출
latitudes = [msg.latitude for msg in rosbag_gps_data['/ublox_gps/fix']['data']]
longitudes = [msg.longitude for msg in rosbag_gps_data['/ublox_gps/fix']['data']]
times = np.array(rosbag_gps_data['/ublox_gps/fix']['time'])
relative_times = times - times[0]

print(f"GPS Fix 좌표 포인트: {len(latitudes)}개")
print(f"위도 범위: {min(latitudes):.8f}° ~ {max(latitudes):.8f}°")
print(f"경도 범위: {min(longitudes):.8f}° ~ {max(longitudes):.8f}°")
print(f"시간 범위: {relative_times[-1]:.1f}초")

# 위성 지도 생성
gps_satellite_map = gnss_processor.create_satellite_map(
    latitudes=latitudes,
    longitudes=longitudes,
    times=relative_times,
    title="GPS Fix Trajectory"
)

if gps_satellite_map:
    print("✓ GPS Fix 위성 지도 생성 완료")
    print("지도를 보려면 다음 셀을 실행하세요:")
    print("gps_satellite_map")
else:
    print("⚠ 위성 지도 생성 실패")

In [ ]:
# 변환할 WGS84 데이터를 위성 지도에 표시
gps_satellite_map

In [ ]:
### GT 데이터로 사용할 ENU 데이터 로드 ###
with open('./../data/gnss/hanyang2_enu_data.txt', 'r') as f:
    gt_enu = np.array([line.strip().split() for line in f.readlines()], dtype=float)

# 데이터 확인
print(gt_enu)

# 데이터 시각화
plt.scatter(gt_enu[:, 0], gt_enu[:, 1], c='b', marker='o')
plt.show()

In [ ]:
# 좌표 변환 실습
print("=== WGS84 to ENU 좌표 변환 실습 ===")

if '/ublox_gps/fix' in rosbag_gps_data and rosbag_gps_data['/ublox_gps/fix']['data']:
    gnss_data = rosbag_gps_data['/ublox_gps/fix']
    
    # 원본 WGS84 좌표 추출
    latitudes = np.array([msg.latitude for msg in gnss_data['data']])
    longitudes = np.array([msg.longitude for msg in gnss_data['data']])
    altitude = np.array([msg.altitude for msg in gnss_data['data']])
    
    # LLH 배열 생성 (Latitude, Longitude, Height)
    original_llh = np.column_stack([xxxxxx, xxxxxx, xxxxxx]) # TODO: 위도, 경도, 고도 추출
    
    # 기준점 설정 (첫 번째 점)
    reference_point = original_llh[0].copy()
    
    print(f"원본 데이터 포인트 수: {len(original_llh)}")
    print(f"기준점: Lat={reference_point[0]:.8f}°, Lon={reference_point[1]:.8f}°, Height={reference_point[2]:.3f}m")

# WGS84 → ENU 변환
print("WGS84 → ENU 변환 중...")
converted_enu = gnss_processor.wgs84_to_enu(original_llh, reference_point) # TODO: wgs84_to_enu 함수 완성
print(f"✓ ENU 좌표 변환 완료")

In [ ]:
# 좌표 변환 결과 비교 및 정확도 분석
print("=== 좌표 변환 정확도 분석 ===")

gnss_processor.plot_wgs84_to_enu_comparison(
    original_llh, converted_enu, gt_enu
)

# WGS84 곡률반지름 계산 및 출력
print(f"\n=== WGS84 곡률반지름 (기준점 기준) ===")
meridional_r = gnss_processor.meridional_radius(reference_point[0])
normal_r = gnss_processor.normal_radius(reference_point[0])

print(f"자오선 곡률반지름 (M): {meridional_r:.3f} m")
print(f"수직 곡률반지름 (N): {normal_r:.3f} m")
print(f"M/N 비율: {meridional_r/normal_r:.6f}")

# ENU 좌표계에서의 거리 분석
print(f"\n=== ENU 좌표계 분석 ===")
east_range = np.max(converted_enu[:, 0]) - np.min(converted_enu[:, 0])
north_range = np.max(converted_enu[:, 1]) - np.min(converted_enu[:, 1])
up_range = np.max(converted_enu[:, 2]) - np.min(converted_enu[:, 2])

print(f"East 방향 범위: {east_range:.3f} m")
print(f"North 방향 범위: {north_range:.3f} m") 
print(f"Up 방향 범위: {up_range:.3f} m")

# 총 이동 거리 계산
total_distance = 0
for i in range(1, len(converted_enu)):
    dx = converted_enu[i, 0] - converted_enu[i-1, 0]
    dy = converted_enu[i, 1] - converted_enu[i-1, 1]
    distance = np.sqrt(dx**2 + dy**2)
    total_distance += distance

print(f"총 수평 이동 거리: {total_distance:.3f} m")

# 3D 그래프 - GT와 변환된 ENU 데이터 비교
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot3D(gt_enu[:, 0], gt_enu[:, 1], gt_enu[:, 2], c='g', marker='o', label='GT ENU', markersize=8)
ax.plot3D(converted_enu[:, 0], converted_enu[:, 1], converted_enu[:, 2], c='r', marker='o', label='Converted ENU', markersize=4)
ax.set_xlabel('E')
ax.set_ylabel('N')
ax.set_zlabel('U')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('ENU to WGS84 Conversion Comparison in 3D')
plt.show()


### 4-2. ENU to WGS84 좌표 변환

취득한 ENU 데이터를 WGS84 좌표계로 변환하는 실습을 진행합니다.

In [ ]:
### ENU 데이터 로드 ###
with open('./../data/gnss/highway_inlet_enu_data.txt', 'r') as f:
    enu_data = np.array([line.strip().split() for line in f.readlines()], dtype=float)

# 데이터 확인
print(enu_data)

# 데이터 시각화
plt.scatter(enu_data[:, 0], enu_data[:, 1], c='b', marker='o')
plt.show()

In [ ]:
### GT 데이터로 사용할 WGS84 데이터 로드 ###
# ROS bag 파일 경로 설정
gt_bag_file_path = "./../data/imu/4_highway_inlet.bag"

# 로드할 토픽 정의
gps_topics = ['/gps/fix']

print(f"파일 경로: {gt_bag_file_path}")
print(f"분석할 토픽: {gps_topics}")

# GPS Fix 데이터 로딩
rosbag_gps_data = gnss_processor.load_rosbag_gnss_data(gt_bag_file_path, gps_topics)

# GPS Fix 데이터에서 궤적 추출
gt_latitudes = [msg.latitude for msg in rosbag_gps_data['/gps/fix']['data']]
gt_longitudes = [msg.longitude for msg in rosbag_gps_data['/gps/fix']['data']]
gt_altitudes = [msg.altitude for msg in rosbag_gps_data['/gps/fix']['data']]
gt_times = np.array(rosbag_gps_data['/gps/fix']['time'])
relative_times = gt_times - gt_times[0]

print(f"GPS Fix 좌표 포인트: {len(gt_latitudes)}개")
print(f"위도 범위: {min(gt_latitudes):.8f}° ~ {max(gt_latitudes):.8f}°")
print(f"경도 범위: {min(gt_longitudes):.8f}° ~ {max(gt_longitudes):.8f}°")
print(f"시간 범위: {relative_times[-1]:.1f}초")

In [ ]:
# 좌표 변환 실습
print("=== ENU to WGS84 좌표 변환 실습 ===")

# GT LLH 배열 생성 (Latitude, Longitude, Height)
gt_llh = np.column_stack([xxxxxx, xxxxxx, xxxxxx]) # TODO: 위도, 경도, 고도 추출
    
# 기준점 설정 (GT의 첫 번째 점)
gt_reference_point = gt_llh[0].copy()

# ENU → WGS84 역변환
print("ENU → WGS84 변환 중...")
converted_wgs84 = gnss_processor.enu_to_wgs84(enu_data, gt_reference_point) # TODO: enu_to_wgs84 함수 완성
print(f"✓ WGS84 좌표 변환 완료")

In [ ]:
# 좌표 변환 결과를 위성 지도에 표시
enu_to_wgs84_satellite_map = gnss_processor.create_satellite_map(
    latitudes=converted_wgs84[:, 0].tolist(),
    longitudes=converted_wgs84[:, 1].tolist(),
    times=relative_times
) # TODO: create_satellite_map 함수 완성

if enu_to_wgs84_satellite_map:
    print("✓ GPS Fix 위성 지도 생성 완료")
    print("지도를 보려면 다음 셀을 실행하세요:")
    print("enu_to_wgs84_satellite_map")
else:
    print("⚠ 위성 지도 생성 실패")

In [ ]:
# 좌표 변환 결과를 위성 지도에 시각화
enu_to_wgs84_satellite_map

In [ ]:
# 좌표 변환 결과 비교 및 정확도 분석
print("=== 좌표 변환 정확도 분석 ===")

gnss_processor.plot_enu_to_wgs84_comparison(
    enu_data, converted_wgs84, gt_llh
)

# WGS84 곡률반지름 계산 및 출력
print(f"\n=== WGS84 곡률반지름 (기준점 기준) ===")
meridional_r = gnss_processor.meridional_radius(gt_reference_point[0])
normal_r = gnss_processor.normal_radius(gt_reference_point[0])

print(f"자오선 곡률반지름 (M): {meridional_r:.3f} m")
print(f"수직 곡률반지름 (N): {normal_r:.3f} m")
print(f"M/N 비율: {meridional_r/normal_r:.6f}")

# WGS84 좌표계에서의 거리 분석
print(f"\n=== WGS84 좌표계 분석 ===")
east_range = np.max(converted_wgs84[:, 0]) - np.min(converted_wgs84[:, 0])
north_range = np.max(converted_wgs84[:, 1]) - np.min(converted_wgs84[:, 1])
up_range = np.max(converted_wgs84[:, 2]) - np.min(converted_wgs84[:, 2])

print(f"East 방향 범위: {east_range:.3f} m")
print(f"North 방향 범위: {north_range:.3f} m") 
print(f"Up 방향 범위: {up_range:.3f} m")

# 총 이동 거리 계산
total_distance = 0
for i in range(1, len(enu_data)):
    dx = enu_data[i, 0] - enu_data[i-1, 0]
    dy = enu_data[i, 1] - enu_data[i-1, 1]
    distance = np.sqrt(dx**2 + dy**2)
    total_distance += distance

print(f"총 수평 이동 거리: {total_distance:.3f} m")

# 3D 그래프 - GT와 변환된 WGS84 데이터 비교
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot3D(gt_llh[:, 1], gt_llh[:, 0], gt_llh[:, 2], c='g', marker='o', label='GT WGS84', markersize=8)
ax.plot3D(converted_wgs84[:, 1], converted_wgs84[:, 0], converted_wgs84[:, 2], c='r', marker='o', label='Converted WGS84', markersize=4)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Height')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('ENU to WGS84 Conversion Comparison in 3D')
plt.show()


## 실습 요약 및 결론

### 🎯 실습에서 배운 내용

1. **NMEA 데이터 분석**
   - GGA와 RMC 메시지 파싱 및 분석
   - HDOP, 위성 수, 속도 등 품질 지표 평가
   - Fix Quality와 Navigation Status 이해

2. **위성 지도 시각화**
   - 실제 위성 지도 위에 GPS 궤적 표시
   - 시간에 따른 색상 변화로 경로 진행 시각화
   - 헤딩 방향 화살표로 차량 방향 표시

3. **실제 취득 GNSS 데이터 분석**
   - GNSS 수신기의 메시지 분석
   - 정확도 지표 확인
   - 다양한 솔루션 타입과 상태 모니터링

4. **좌표 변환 (WGS84 ↔ ENU)**
   - 전역 좌표계와 지역 좌표계 간 변환 구현
   - 지구 타원체 곡률반지름 계산
   - 변환 정확도 검증 및 오차 분석

### 💡 자율주행에서의 GNSS 활용

- **위치 인식**: 차량의 절대적 위치 파악으로 지도 기반 항법
- **경로 계획**: 목적지까지의 글로벌 경로 생성
- **센서 퓨전**: 카메라, 라이다 등과 결합하여 정확도 향상
- **안전 시스템**: GPS 기반 지오펜싱 및 위험 구역 회피
- **V2X 통신**: 다른 차량 및 인프라와 위치 정보 공유

### 🔧 GNSS 데이터 품질 평가 기준

1. **정확도 지표**
   - HDOP < 2.0: 우수한 정확도
   - Standard Deviation < 1m: 일반 GPS 수준
   - Standard Deviation < 0.1m: 고정밀 RTK 수준

2. **연속성 평가**
   - Fix Quality 지속성
   - 위성 수 안정성 (8개 이상 권장)
   - 신호 끊김 빈도

### 📊 좌표 변환의 중요성

- **지역적 직관성**: 미터 단위 상대 거리로 직관적 이해
- **계산 효율성**: 평면 좌표계에서 빠른 거리/각도 계산
- **센서 통합**: 차량 중심 좌표계에서 통합 처리
- **제어 시스템**: 실시간 차량 제어에 최적화된 좌표계